In [1]:
!pip install mxnet
!pip install gluonnlp==0.9.1
!pip install sentencepiece
!pip install transformers
!pip install torch

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.1/49.1 MB 28.0 MB/s eta 0:00:0000:0100:01
  Attempting uninstall: graphviz
    Found existing installation: graphviz 0.20.1
    Uninstalling graphviz-0.20.1:
      Successfully uninstalled graphviz-0.20.1
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 252.8/252.8 kB 18.6 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for gluonnlp: filename=gluonnlp-0.9.1-cp39-cp39-linux_x86_64.whl size=362442 sha256=bb952b5db676d18435ca9c2b1274787ee5b8ed8363925beca0fbdefc4f890e3f
  Stored in directory: /home/studio/.cache/pip/wheels/f0/1a/57/1fac92705aa8b074509a0f514fc0cb749ca6332feb3901692d
Successfully built gluonnlp


In [1]:
from torch.utils.data import Dataset, DataLoader
import torch
from torch import nn
import torch.nn.functional as F
import torch.optim as optim

import gluonnlp as nlp

import numpy as np
import pandas as pd

import torch.nn.functional as F
import torch.optim as optim
from tqdm.notebook import tqdm

from transformers import AdamW
from transformers.optimization import get_cosine_schedule_with_warmup

In [3]:
pip install 'git+https://github.com/SKTBrain/KoBERT.git#egg=kobert_tokenizer&subdirectory=kobert_hf'

  Cloning https://github.com/SKTBrain/KoBERT.git to /tmp/pip-install-pmirqvou/kobert-tokenizer_c3354c6bfdb3441f82b50dc6795aef74
  Running command git clone --filter=blob:none --quiet https://github.com/SKTBrain/KoBERT.git /tmp/pip-install-pmirqvou/kobert-tokenizer_c3354c6bfdb3441f82b50dc6795aef74
  Resolved https://github.com/SKTBrain/KoBERT.git to commit 47a69af87928fc24e20f571fe10c3cc9dd9af9a3
  Preparing metadata (setup.py) ... done
  Created wheel for kobert_tokenizer: filename=kobert_tokenizer-0.1-py3-none-any.whl size=4648 sha256=eb77a1b19e874462685e10c0dbcdf2eabaabf9eec0106c561d766409972679b2
  Stored in directory: /tmp/pip-ephem-wheel-cache-g2bfr_n4/wheels/64/c4/82/d70d864682468fad0bf8269b0dfd88daf48f209be4c8ade164
Successfully built kobert_tokenizer
Note: you may need to restart the kernel to use updated packages.


In [2]:
from kobert_tokenizer import KoBERTTokenizer
import torch
tokenizer = KoBERTTokenizer.from_pretrained('skt/kobert-base-v1', sp_model_kwargs={'nbest_size': -1, 'alpha': 0.6, 'enable_sampling': True})
tokenizer.encode("한국어 모델을 공유합니다.")

The tokenizer class you load from this checkpoint is not the same type as the class this function is called from. It may result in unexpected tokenization. 
The tokenizer class you load from this checkpoint is 'XLNetTokenizer'. 
The class this function is called from is 'KoBERTTokenizer'.


[2, 4958, 6855, 2046, 7088, 1050, 7843, 54, 3]

In [20]:
from transformers import BertModel
model = BertModel.from_pretrained('skt/kobert-base-v1')
text = "한국어 모델을 공유합니다."
inputs = tokenizer.batch_encode_plus([text])
out = model(input_ids = torch.tensor(inputs['input_ids']),
              attention_mask = torch.tensor(inputs['attention_mask']))
out.pooler_output.shape

torch.Size([1, 768])

In [6]:
model

BertModel(
  (embeddings): BertEmbeddings(
    (word_embeddings): Embedding(8002, 768, padding_idx=1)
    (position_embeddings): Embedding(512, 768)
    (token_type_embeddings): Embedding(2, 768)
    (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
    (dropout): Dropout(p=0.1, inplace=False)
  )
  (encoder): BertEncoder(
    (layer): ModuleList(
      (0-11): 12 x BertLayer(
        (attention): BertAttention(
          (self): BertSelfAttention(
            (query): Linear(in_features=768, out_features=768, bias=True)
            (key): Linear(in_features=768, out_features=768, bias=True)
            (value): Linear(in_features=768, out_features=768, bias=True)
            (dropout): Dropout(p=0.1, inplace=False)
          )
          (output): BertSelfOutput(
            (dense): Linear(in_features=768, out_features=768, bias=True)
            (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
            (dropout): Dropout(p=0.1, inplace=False)
   

In [21]:
tokenizer = KoBERTTokenizer.from_pretrained('skt/kobert-base-v1')
bertmodel = BertModel.from_pretrained('skt/kobert-base-v1', return_dict=False)
vocab = nlp.vocab.BERTVocab.from_sentencepiece(tokenizer.vocab_file, padding_token='[PAD]')

The tokenizer class you load from this checkpoint is not the same type as the class this function is called from. It may result in unexpected tokenization. 
The tokenizer class you load from this checkpoint is 'XLNetTokenizer'. 
The class this function is called from is 'KoBERTTokenizer'.


In [22]:
device = torch.device("cuda:0")

In [12]:
!wget https://www.dropbox.com/s/374ftkec978br3d/ratings_train.txt?dl=1
!wget https://www.dropbox.com/s/977gbwh542gdy94/ratings_test.txt?dl=1

--2023-07-13 00:42:52--  https://www.dropbox.com/s/374ftkec978br3d/ratings_train.txt?dl=1
www.dropbox.com (www.dropbox.com) 해석 중... 162.125.84.18, 2620:100:6034:18::a27d:5412
다음으로 연결 중: www.dropbox.com (www.dropbox.com)|162.125.84.18|:443... 연결했습니다.
HTTP 요청을 보냈습니다. 응답 기다리는 중... 302 Found
위치: /s/dl/374ftkec978br3d/ratings_train.txt [따라감]
--2023-07-13 00:42:52--  https://www.dropbox.com/s/dl/374ftkec978br3d/ratings_train.txt
www.dropbox.com에 기존 연결 재활용:443.
HTTP 요청을 보냈습니다. 응답 기다리는 중... 302 Found
위치: https://uc0315e0bfe5c1579e1beea6197a.dl.dropboxusercontent.com/cd/0/get/B_tdnOk2XjqOwtZSyap6EePz8E5Qx339EuWf0mHyOn97flp26cDWZ3TG_Ucz6sTnji66nldSC2BmJgL_9BFPGZ8zDnb_h0uZn7joulBwDDH4Bs4qpWMahqwgERIkanKgS0yS_H-8WHvZ3LFd-Vs6y95o7gs5XRNVq-WVLomWmE2bzg/file?dl=1# [따라감]
--2023-07-13 00:42:52--  https://uc0315e0bfe5c1579e1beea6197a.dl.dropboxusercontent.com/cd/0/get/B_tdnOk2XjqOwtZSyap6EePz8E5Qx339EuWf0mHyOn97flp26cDWZ3TG_Ucz6sTnji66nldSC2BmJgL_9BFPGZ8zDnb_h0uZn7joulBwDDH4Bs4qpWMahqwgERIkanKgS0yS_H-8W

In [13]:
dataset_train = nlp.data.TSVDataset("ratings_train.txt?dl=1", field_indices=[1,2], num_discard_samples=1)
dataset_test = nlp.data.TSVDataset("ratings_test.txt?dl=1", field_indices=[1,2], num_discard_samples=1)

In [17]:
class BERTDataset(Dataset):
    def __init__(self, dataset, sent_idx, label_idx, bert_tokenizer, vocab, max_len,
                 pad, pair):
        transform = nlp.data.BERTSentenceTransform(
            bert_tokenizer, max_seq_length=max_len, vocab=vocab, pad=pad, pair=pair)

        self.sentences = [transform([i[sent_idx]]) for i in dataset]
        self.labels = [np.int32(i[label_idx]) for i in dataset]

    def __getitem__(self, i):
        return (self.sentences[i] + (self.labels[i], ))

    def __len__(self):
        return (len(self.labels))

In [25]:
max_len = 64
batch_size = 64
warmup_ratio = 0.1
num_epochs = 5
max_grad_norm = 1
log_interval = 200
learning_rate =  5e-5

In [23]:
tok = tokenizer.tokenize

data_train = BERTDataset(dataset_train, 0, 1, tok, vocab, max_len, True, False)
data_test = BERTDataset(dataset_test, 0, 1, tok, vocab, max_len, True, False)

In [18]:
train_dataloader = torch.utils.data.DataLoader(data_train, batch_size=batch_size, num_workers=5)
test_dataloader = torch.utils.data.DataLoader(data_test, batch_size=batch_size, num_workers=5)

In [4]:
class BERTClassifier(nn.Module):
    def __init__(self,
                 bert,
                 hidden_size = 768,
                 num_classes=2,
                 dr_rate=None,
                 params=None):
        super(BERTClassifier, self).__init__()
        self.bert = bert
        self.dr_rate = dr_rate
                 
        self.classifier = nn.Linear(hidden_size , num_classes)
        if dr_rate:
            self.dropout = nn.Dropout(p=dr_rate)
    
    def gen_attention_mask(self, token_ids, valid_length):
        attention_mask = torch.zeros_like(token_ids)
        for i, v in enumerate(valid_length):
            attention_mask[i][:v] = 1
        return attention_mask.float()

    def forward(self, token_ids, valid_length, segment_ids):
        attention_mask = self.gen_attention_mask(token_ids, valid_length)
        
        _, pooler = self.bert(input_ids = token_ids, token_type_ids = segment_ids.long(), attention_mask = attention_mask.float().to(token_ids.device))
        if self.dr_rate:
            out = self.dropout(pooler)
        return self.classifier(out)

In [20]:
model = BERTClassifier(bertmodel,  dr_rate=0.5).to(device)

In [21]:
no_decay = ['bias', 'LayerNorm.weight']
optimizer_grouped_parameters = [
    {'params': [p for n, p in model.named_parameters() if not any(nd in n for nd in no_decay)], 'weight_decay': 0.01},
    {'params': [p for n, p in model.named_parameters() if any(nd in n for nd in no_decay)], 'weight_decay': 0.0}
]

In [22]:
optimizer = AdamW(optimizer_grouped_parameters, lr=learning_rate)
loss_fn = nn.CrossEntropyLoss()

t_total = len(train_dataloader) * num_epochs
warmup_step = int(t_total * warmup_ratio)

scheduler = get_cosine_schedule_with_warmup(optimizer, num_warmup_steps=warmup_step, num_training_steps=t_total)

/home/studio/anaconda3/envs/project/lib/python3.9/site-packages/transformers/optimization.py:391: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


In [23]:
def calc_accuracy(X,Y):
    max_vals, max_indices = torch.max(X, 1)
    train_acc = (max_indices == Y).sum().data.cpu().numpy()/max_indices.size()[0]
    return train_acc

In [26]:
for e in range(num_epochs):
    train_acc = 0.0
    test_acc = 0.0
    model.train()
    for batch_id, (token_ids, valid_length, segment_ids, label) in enumerate(tqdm(train_dataloader)):
        optimizer.zero_grad()
        token_ids = token_ids.long().to(device)
        segment_ids = segment_ids.long().to(device)
        valid_length= valid_length
        label = label.long().to(device)
        out = model(token_ids, valid_length, segment_ids)
        loss = loss_fn(out, label)
        loss.backward()
        torch.nn.utils.clip_grad_norm_(model.parameters(), max_grad_norm)
        optimizer.step()
        scheduler.step()  # Update learning rate schedule
        train_acc += calc_accuracy(out, label)
        if batch_id % log_interval == 0:
            print("epoch {} batch id {} loss {} train acc {}".format(e+1, batch_id+1, loss.data.cpu().numpy(), train_acc / (batch_id+1)))
    print("epoch {} train acc {}".format(e+1, train_acc / (batch_id+1)))
    model.eval()
    for batch_id, (token_ids, valid_length, segment_ids, label) in enumerate(tqdm(test_dataloader)):
        token_ids = token_ids.long().to(device)
        segment_ids = segment_ids.long().to(device)
        valid_length= valid_length
        label = label.long().to(device)
        out = model(token_ids, valid_length, segment_ids)
        test_acc += calc_accuracy(out, label)
    print("epoch {} test acc {}".format(e+1, test_acc / (batch_id+1)))

  0%|          | 0/2344 [00:00<?, ?it/s]

epoch 1 batch id 1 loss 0.6927973628044128 train acc 0.640625
epoch 1 batch id 201 loss 0.5581398606300354 train acc 0.5772699004975125
epoch 1 batch id 401 loss 0.4007514715194702 train acc 0.6817721321695761
epoch 1 batch id 601 loss 0.4294225871562958 train acc 0.733205074875208
epoch 1 batch id 801 loss 0.4648691713809967 train acc 0.7625624219725343
epoch 1 batch id 1001 loss 0.28572356700897217 train acc 0.7798607642357642
epoch 1 batch id 1201 loss 0.30930984020233154 train acc 0.7932452123230641
epoch 1 batch id 1401 loss 0.3249175548553467 train acc 0.801960653104925
epoch 1 batch id 1601 loss 0.28079214692115784 train acc 0.8102943472829481
epoch 1 batch id 1801 loss 0.2638770341873169 train acc 0.816777137701277
epoch 1 batch id 2001 loss 0.2757550776004791 train acc 0.8224481509245377
epoch 1 batch id 2201 loss 0.29596811532974243 train acc 0.8274363925488414
epoch 1 train acc 0.8309913609215017


  0%|          | 0/782 [00:00<?, ?it/s]

epoch 1 test acc 0.8860294117647058


  0%|          | 0/2344 [00:00<?, ?it/s]

...

epoch 4 test acc 0.897258631713555


  0%|          | 0/2344 [00:00<?, ?it/s]

epoch 5 batch id 1 loss 0.3733232021331787 train acc 0.890625
epoch 5 batch id 201 loss 0.02429956942796707 train acc 0.9745802238805971
epoch 5 batch id 401 loss 0.10971395671367645 train acc 0.9757247506234414
epoch 5 batch id 601 loss 0.12054506689310074 train acc 0.9763415141430949
epoch 5 batch id 801 loss 0.19424889981746674 train acc 0.9765917602996255
epoch 5 batch id 1001 loss 0.011286986991763115 train acc 0.977662962037962
epoch 5 batch id 1201 loss 0.027933500707149506 train acc 0.9782212739383847
epoch 5 batch id 1401 loss 0.07862568646669388 train acc 0.9783636688079943
epoch 5 batch id 1601 loss 0.05443156510591507 train acc 0.9788803872579638
epoch 5 batch id 1801 loss 0.008495762944221497 train acc 0.9792216129927818
epoch 5 batch id 2001 loss 0.007236095145344734 train acc 0.9794946276861569
epoch 5 batch id 2201 loss 0.09542693942785263 train acc 0.9796328373466606
epoch 5 train acc 0.9797354948805461


  0%|          | 0/782 [00:00<?, ?it/s]

epoch 5 test acc 0.8973385549872123


In [27]:
torch.save(model,'model.pt') 

In [5]:
test_model = torch.load('model.pt')

In [7]:
def predict(predict_sentence):
    data = [predict_sentence, '0']
    dataset_another = [data]

    another_test = BERTDataset(dataset_another, 0, 1, tok, vocab, max_len, True, False)
    test_dataloader = torch.utils.data.DataLoader(another_test, batch_size=batch_size, num_workers=5)
    
    test_model.eval()

    for batch_id, (token_ids, valid_length, segment_ids, label) in enumerate(test_dataloader):
        token_ids = token_ids.long().to(device)
        segment_ids = segment_ids.long().to(device)

        valid_length= valid_length
        label = label.long().to(device)

        out = test_model(token_ids, valid_length, segment_ids)

        test_eval=[]
        for i in out:
            logits=i
            logits = logits.detach().cpu().numpy()

            if np.argmax(logits) == 0:
                test_eval.append("부정")
            elif np.argmax(logits) == 1:
                test_eval.append("긍정")
            else:
                test_eval.append("??")

        print(">> 입력하신 내용에서 " + test_eval[0] + " 느껴집니다.")

In [38]:
mbti = pd.read_csv('mbti_final_data.csv')

In [39]:
from ast import literal_eval
text = literal_eval(mbti['contents'][0])

In [40]:
text[0]

'istj 현실주의자 conclusion 결론 결론 현실주의자만큼 실용적이고 헌신적인 유형은 거의 없습니다. 신뢰성과 노력으로 유명한 현실주의자는 자신과 사랑하는 사람들을 위해 안전하고 안정적인 환경을 형성하고 유지하는 것에 능숙합니다. 현실주의자들의 헌신은 자기계발을 포함한 많은 영역에서 중요하게 활용됩니다. 하지만 현실주의자들의 실용적이고 체계적인 접근방식이 오히려 문제가 되는 상황도 생길 수 있습니다. 연애상대를 찾고 관계를 유지하는 것이나 긴장을 줄이고 융통성있게 배우는 것이나 업무량을 관리하는 것 등 현실주의자들은 스스로의 약점을 극복하고 추가적인 역량 강화를 위해 의식적인 노력을 기울여야 합니다. istj 현실주의자 introduction 소개 포스팅으로 이동'

In [41]:
from konlpy.tag import Kkma
kkma = Kkma()
test = kkma.sentences(text[1])

In [50]:
# test
kkma.sentences(test[-6][:20])

['동시에 규칙에 따르는 것을 선호하고']

In [27]:
test

['istj 현실주의자 workplace habits 직장 습관 직장 습관 현실주의자는 직장에서 전형적인 근면하고 성실한 직원입니다.',
 '어떤 직책에서도 현실주의자들 유형은 명확하게 정의된 규칙과 권위 및 직책에 대한 존중을 추구합니다.',
 '현실주의자에게 책임은 부담이 아니라 신뢰이며 일에 적합한 사람이라는 것을 증명할 수 있는 기회입니다.',
 '반면에 새로운 책임을 떠안고 반대로 오래된 책임을 내려놓는 것에 수반되는 변화는 현실주의자들이 어렵게 느끼는 부분 중 하나입니다.',
 '어떤 직책에 있느냐에 따라 다르게 나타나지만 이들이 극복해야 할 가장 중요한 과제라고 할 수 있습니다.',
 '현실주의자 유형의 많은 사람들은 개인적이고 전문적인 업무에 집중하는 경우가 많습니다.',
 '현실주의자 직원 현실주의자들이 책임을 중요시하는 성향은 실제 중요하지 않은 여러 업무를 떠안게 만드는 요인으로 작용하기도 합니다.',
 '이 들은 모든 분야에서 전문가로 간주되어 메뉴 얼이 정해진 프로젝트를 아주 능숙하게 처리할 수 있는 능력을 가지고 있습니다.',
 '하지만 과도한 부담감을 느끼거나 일에 더욱 적합한 사람이 있는 경우에도 책임을 포기하는 것을 꺼리게 만드는 경우도 있습니다.',
 '현실주의자의 업무에 대한 진지함은 비판에 매우 예민 해지게 만들고 업무를 수행하는 과정에서 짜증이 날 수도 있습니다.',
 '현실주의자는 이처럼 업무에 진지하기 때문에 가장 생산 적인 직원이 될 수 있습니다.',
 '권위와 위계를 중시하고 명령과 지시를 따르는 것이 뛰어나기 때문입니다.',
 '이들이 시간을 엄수하고 정시에 출근하고 프로젝트 마감일을 맞추는 것에 문제가 생길 일은 없습니다.',
 '명확하게 정해진 업무체계와 책임이 함께 있다면 현실주의자들은 매우 충성스럽고 헌신적이며 꼼꼼 하게 일을 추진해 나갑니다.',
 '현실주의자 동료 동료들 사이에서 현실주의자 만큼 프로젝트를 제시간에 메뉴 얼대로 끝나는 것을 보장할 수 있는 믿음 직 스러 운 사람은 없습니다.',
 '조용하

In [54]:
F.softmax(out,dim=1)

tensor([[0.9986, 0.0014]], device='cuda:0', grad_fn=<SoftmaxBackward0>)